In [1]:
import pandas as pd
import selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
#from urllib.request import urlopen
#import urllib
from selenium.webdriver.support.ui import Select
import winsound
from PIL import Image
import time

In [2]:
keyword = "骨質疏鬆"
agency = "國民健康署"
startDate = "2000-01-01"
endDate = "2024-08-09"
page = 1

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())  #使用webdriver開啟chrome
url = f"https://health99.hpa.gov.tw/search?keyword={keyword}&startDate={startDate}&endDate={endDate}&tab=1&materialType=&releaseType=&releaseAgency={agency}&ask=&sort=&page={page}"
driver.get(url) 

In [4]:
def get_data_page(page):
    url = f"https://health99.hpa.gov.tw/search?keyword={keyword}&startDate=2000-01-01&endDate=2024-06-27&tab=1&materialType=&releaseType=&releaseAgency={agency}&ask=&sort=&page={page}"
    driver.get(url) 
    time.sleep(5)
    data_eles = driver.find_elements(By.CLASS_NAME,"col-md-2.col-sm-3.col-xs-4.px-8.mb-4.show_photo")
    data =[]
    for element in data_eles:
        text = element.text
        tag = text.split("\n")[0]
        title = text.split("\n")[1]
        page_url = element.get_attribute("href")
        img_ele = element.find_element(By.TAG_NAME, 'img')
        image_url = img_ele.get_attribute('src')
        # print(tag)
        print(title)
        # print(page_url)
        # print(image_url)
        data.append({
            "tag":tag,
            "title":title,
            "page_url":page_url,
            "image_url":image_url
        })
    return(pd.DataFrame(data))

In [18]:
df_list = []
page = 1
while True:
    try:
        page_df = get_data_page(page)
        if len(page_df)==0:
            break
        df_list.append(page_df)
        page = page + 1
    except:
        break

【教案】買進漲停「骨」，補「鈣」第一步
第3堂：擔心骨鬆找上你，教你一起堅固骨骼
醫病共享決策-預防骨質疏鬆，您想藥什麼?
活躍肌肉-預防肌少症(國語)
骨質疏鬆是老人病？30歲後你也可能是高危險群！
如何預防骨質疏鬆
預防骨質疏鬆海報
預防骨質疏鬆
骨質疏鬆症臨床治療指引手冊
認識骨質密度檢測
Taiwan Osteoporosis Practice Guidelines 骨質疏鬆症臨床治療指引(英文版)


In [19]:
df = pd.concat(df_list,axis=0)
df.reset_index(drop=True,inplace=True)
for i in range(len(df)):
    page_url = df.loc[i,"page_url"]
    driver.get(page_url)
    li_element_date = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, "//li[contains(text(), '發佈日期')]"))
                )
    date = li_element_date.text.split("：")[1]
    li_element_author = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, "//li[contains(text(), '作者')]"))
                )
    author = li_element_author.text.split("：")[1]
    df.loc[i,"date"] = date
    df.loc[i,"author"] = author

In [20]:
df.to_csv(f"全部{keyword}教材列表整理.csv",encoding="utf-8-sig",index=False)
df.loc[df["author"]=="慢性疾病防治組",].sort_values(by=["date"]).to_csv(f"本組{keyword}教材列表整理.csv",encoding="utf-8-sig",index=False)